In [1]:
import pandas as pd
import numpy as np

In [2]:
#Input data
S_OLD_18=392945800 #surface of existing buildings in 2018 (residential)
Dwe_avg=6 #average dwellings built per 1000 inhabitant
S_avg_dwe=99 #average surface per dwelling
Population_18=8525611 #Swiss population in 2018
Population_50=10440600 #Swiss population in 2050

EM_OP_OLD=28 #average CO2 emissions operation of existing buildings (2018) in kgCO2eq/m2.year

years = [] #temporal frame of study 2018 - 2050
for i in range (2018, 2051, 1):
    years.append(i)

In [3]:
#Variable parameters

Goal_y = [2040, 2045, 2050]

REN_rate = [0.01, 0.03, 0.05, 0.1] #possible renovation rates
    
Op_NEW=[0,1,2,3,4,5,6,7,8,9,10] #in kgCO2/m2.year - new buildings
Op_REN=[0,1,2,3,4,5,6,7,8,9,10] #in kgCO2/m2.year - renovations

#possible embodied emissions in kgCO2eq/m2 
EM_EMB_REN=[] #for renovation works
EM_EMB_NEW=[] #for new buildings

for i in range (-300, 601, 60):
    EM_EMB_REN.append(i)
    
for i in range (-540, 1200, 120):
    EM_EMB_NEW.append(i)


In [18]:
#Dynamic evolution of variable parameters - linear till goal year,
#These funtions return values for specific year and variable chosen

def Ren_rate (year, REN_rate, Goal_y): #dynamic evolution of renovation rate till 2050
    return (((REN_rate-0.01)/(Goal_y-2018))*(year - 2018))+0.01 #1% is the initial renovation rate in 2018


def EM_OP_REN (year, Op_REN, Goal_y):
    return (((Op_REN-5.8)/(Goal_y-2018))*(year - 2018))+5.8 #5kgCO2eq/m2.year is considerent the current value of emissions after renovation
    
def EM_OP_NEW (year, Op_NEW, Goal_y):
    return (((Op_NEW-3.5)/(Goal_y-2018))*(year - 2018))+3.5   #2kgCO2eq/m2.year is considerent the current value of emissions of new buildings

def EMB_REN (year, EM_EMB_REN, Goal_y):
    return (((EM_EMB_REN-440)/(Goal_y-2018))*(year - 2018))+440 #440kgCO2eq/m2 - current level of emissions for renovation
    
def EMB_NEW (year, EM_EMB_NEW, Goal_y):
    return (((EM_EMB_NEW-696)/(Goal_y-2018))*(year - 2018))+696 #696kgCO2eq/m2 - current level of emissions for new


In [19]:
#Definition of evolution of surfaces

#Renovated surfaces consider the initial stock (surfaces of 2018 in input) and multiply it by the renovation rate of the corresponding year (considerin evolution of renovation rate over time)
#Renovated surfaces are here cumulated over the years (as the operational emissions are on all surfaces every year)
#As renovated surfaces refer to the stock of 2018 - once all surfaces are renovated - no more renovations are taking place in this time frame (buildings built after 2018 will not undergo major renovation works before 2050)
def Surf_REN_cumul (year, S_OLD_18, Ren_rate, REN_rate, Goal_y):
    TRY=[]
    for i in range (years[0], year+1, 1):
        Surfaces_renovated=(S_OLD_18*Ren_rate(i, REN_rate, Goal_y))
        TRY.append(Surfaces_renovated)
    Cumulative=sum(TRY)
    if Cumulative < S_OLD_18:
            return Cumulative
    else:
            return S_OLD_18

#in order to calculate the embodied emissions of renovations only surfaces afected in that year must be considered:        
def Surf_REN_year (year, Surf_REN_cumul,S_OLD_18, Ren_rate, REN_rate, Goal_y):
    return (Surf_REN_cumul(year, S_OLD_18, Ren_rate, REN_rate, Goal_y))- (Surf_REN_cumul(year-1,S_OLD_18, Ren_rate, REN_rate, Goal_y))

#old surfaces decrease according to the renovated surfaces
def Surf_OLD (year, S_OLD_18, Surf_REN_cumul, Ren_rate, REN_rate, Goal_y):
    return S_OLD_18-(Surf_REN_cumul(year, S_OLD_18, Ren_rate, REN_rate, Goal_y))

#New surfaces are calculated as a proportion to the population increase
def Population (year, Population_18, Population_50):
    return ((Population_50-Population_18)/(2050-2018))*(year-2018)+Population_18

#current number and size of dwellings per capita are used
def Surf_NEW (year, Population, S_avg_dwe, Dwe_avg):
    return (Population/1000)*(Dwe_avg*S_avg_dwe)

#similar to renovated surfaces, new surfaces are cumulated over the years in order to account for total operational emissions
def Surf_NEW_cumul (year, Population, S_avg_dwe, Dwe_avg):
    NEW=[]
    for i in range (years[0], year+1, 1):
        New_surfaces= (Population (i, Population_18, Population_50)/1000)*(Dwe_avg*S_avg_dwe)
        NEW.append(New_surfaces)
    Cumulative_NEW=sum(NEW)
    return Cumulative_NEW


In [25]:
#Definition of emissions
def Emissions_OP (year, Surf_OLD, Surf_REN_cumul, Surf_NEW_cumul, EM_OP_REN, EM_OP_NEW):
    return (Surf_OLD*(EM_OP_OLD/1000000000))+((Surf_REN_cumul*(EM_OP_REN/1000000000)))+ (Surf_NEW_cumul*(EM_OP_NEW/1000000000))

def Emissions_EMB (year, Surf_REN_year, Surf_NEW, EMB_REN, EMB_NEW):
    return (Surf_REN_year*(EMB_REN/1000000000))+(Surf_NEW*(EMB_NEW/1000000000))

In [28]:
#cumulative operational emissions

def Cumulative_OP(Surf_OLD, Surf_REN_cumul, Surf_NEW_cumul, EM_OP_REN, EM_OP_NEW, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, Op_REN, Op_NEW, Goal_y):
    Cumul = []
    for i in range (years[0], years[-1]+1, 1):
        Cumul.append(Emissions_OP(i, Surf_OLD(i, S_OLD_18, Surf_REN_cumul, Ren_rate, REN_rate, Goal_y), Surf_REN_cumul(i, S_OLD_18, Ren_rate, REN_rate, Goal_y), Surf_NEW_cumul(i, Population, S_avg_dwe, Dwe_avg), EM_OP_REN(i, Op_REN, Goal_y), EM_OP_NEW(i, Op_NEW, Goal_y)))
    return sum(Cumul)

205.68285818509554

In [29]:
#cumulative embodied emissions


def Cumulative_EMB(Surf_REN_year, Surf_NEW, EMB_REN, EMB_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, EM_EMB_REN, EM_EMB_NEW, Population_18, Population_50, Goal_y):
    Cumuli = []
    for i in range (years[0], years[-1]+1, 1):
        Cumuli.append(Emissions_EMB(i, Surf_REN_year(i, Surf_REN_cumul,S_OLD_18, Ren_rate, REN_rate, Goal_y), Surf_NEW(i, Population(i, Population_18, Population_50), S_avg_dwe, Dwe_avg), EMB_REN(i, EM_EMB_REN, Goal_y), EMB_NEW(i, EM_EMB_NEW, Goal_y)))
    return sum(Cumuli)

239.55022836733315

In [30]:
#Cumulative total emissions operational + embodied

def Cumulative_TOT(Surf_OLD, Surf_REN_year, Surf_NEW, Surf_NEW_cumul, EMB_REN, EMB_NEW, EM_OP_REN, EM_OP_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, EM_EMB_REN, EM_EMB_NEW, Op_REN, Op_NEW, Population_18, Population_50, Goal_y):
    Cumula = []
    for i in range (years[0], years[-1]+1, 1):
        Cumula.append(Emissions_EMB(i, Surf_REN_year(i, Surf_REN_cumul,S_OLD_18, Ren_rate, REN_rate, Goal_y), Surf_NEW(i, Population(i, Population_18, Population_50), S_avg_dwe, Dwe_avg), EMB_REN(i, EM_EMB_REN, Goal_y), EMB_NEW(i, EM_EMB_NEW, Goal_y))+Emissions_OP(i, Surf_OLD(i, S_OLD_18, Surf_REN_cumul, Ren_rate, REN_rate, Goal_y), Surf_REN_cumul(i, S_OLD_18, Ren_rate, REN_rate, Goal_y), Surf_NEW_cumul(i, Population, S_avg_dwe, Dwe_avg), EM_OP_REN(i, Op_REN, Goal_y), EM_OP_NEW(i, Op_NEW, Goal_y)))
    return sum(Cumula)

583.9132425638394

In [31]:
#create csv with all combinations to plot after
result_cum_op=[]

import csv

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Goal_y', 'OP_NEW', 'OP_REN', 'EM_REN', 'EM_NEW', 'REN_RATE', 'Operational_Emissions', 'Embodied_emissions', 'Total emissions'])
    for a in range(len(Goal_y)):
        result_cum_op.append([])          
        for b in range(Op_NEW[0], Op_NEW[-1]+1, 1):
            result_cum_op[a].append([])
            for c in range(Op_REN[0], Op_REN[-1]+1, 1):
                result_cum_op[a][b].append([])
                for d in range(len(EM_EMB_REN)):
                    result_cum_op[a][b][c].append([])
                    for e in range(len(EM_EMB_NEW)):
                        result_cum_op[a][b][c][d].append([])
                        for f in range(len(REN_rate)):
                            if f == 0:
                                ff=0.01
                            elif f == 1:
                                ff=0.03
                            elif f == 2:
                                ff=0.05
                            elif f == 3:
                                ff=0.1
                                
                            aa = a*5+2040

                            dd = d*60-300

                            ee=e*120-540
                        
#Cumulative_OP(Surf_OLD, Surf_REN_cumul, Surf_NEW_cumul, EM_OP_OLD, EM_OP_REN, EM_OP_NEW, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, Op_REN, Op_NEW)
                        
                            result_cum_op[a][b][c][d][e].append(Cumulative_OP(Surf_OLD, Surf_REN_cumul, Surf_NEW_cumul, EM_OP_REN, EM_OP_NEW, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, c, b, aa))
                            for g in range (len(result_cum_op)):
#Cumulative_EMB(Surf_REN_year, Surf_NEW, EMB_REN, EMB_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, EM_EMB_REN, EM_EMB_NEW, Population_18, Population_50)
                                result_cum_op[a][b][c][d][e].append(Cumulative_EMB(Surf_REN_year, Surf_NEW, EMB_REN, EMB_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, dd, ee, Population_18, Population_50, aa))
                                for h in range (len(result_cum_op)):
#Cumulative_TOT(Surf_OLD, Surf_REN_year, Surf_NEW, Surf_NEW_cumul, EMB_REN, EMB_NEW, EM_OP_OLD, EM_OP_REN, EM_OP_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, REN_rate, Population, S_avg_dwe, Dwe_avg, EM_EMB_REN, EM_EMB_NEW, Op_REN, Op_NEW, Population_18, Population_50)
                                    result_cum_op[a][b][c][d][e].append(Cumulative_TOT(Surf_OLD, Surf_REN_year, Surf_NEW, Surf_NEW_cumul, EMB_REN, EMB_NEW, EM_OP_REN, EM_OP_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, dd, ee, c, b, Population_18, Population_50, aa))
                            writer.writerow([aa, b, c, dd, ee, ff, Cumulative_OP(Surf_OLD, Surf_REN_cumul, Surf_NEW_cumul, EM_OP_REN, EM_OP_NEW, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, c, b, aa), Cumulative_EMB(Surf_REN_year, Surf_NEW, EMB_REN, EMB_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, dd, ee, Population_18, Population_50, aa), Cumulative_TOT(Surf_OLD, Surf_REN_year, Surf_NEW, Surf_NEW_cumul, EMB_REN, EMB_NEW, EM_OP_REN, EM_OP_NEW, Surf_REN_cumul, S_OLD_18, Ren_rate, ff, Population, S_avg_dwe, Dwe_avg, dd, ee, c, b, Population_18, Population_50, aa)])
                        